In [2]:
# PERMITE QUE NO COLAB ACESSE O GOOGLE DRIVE
from google.colab import files


**BAIXANDO E CONFIGURANDO O SPARK**

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


tar: Error opening archive: Failed to open 'spark-3.1.2-bin-hadoop2.7.tgz'


In [5]:
!apt-get install openjdk-8-jdk-headless -qq

'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


**Variáveis de ambiente para o Spark e Java**

In [6]:
import os,requests

In [7]:
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [8]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

**Configurar e intalar o findspark**

In [9]:
!pip install -q findspark

In [10]:
import findspark

In [11]:
findspark.init()

Exception: Unable to find py4j in /content/spark-3.1.2-bin-hadoop2.7\python, your SPARK_HOME may not be configured correctly

**Importando bibliotecas**

In [11]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, when, round

In [12]:
# iniciando sessão do spark
spark = SparkSession.builder.appName("Homicidios_Analise").getOrCreate()

# Criando pasta "Base_de_Dados" casos ela não exista
os.makedirs("/content/Base_de_dados", exist_ok=True)
caminho_pasta = "/content/Base_de_dados"

**URL do arquivo no drive**

In [13]:
url = "https://drive.google.com/uc?id=1SL7Wax7TjJagLwObixeMGbfcKHXpn_VR"

**Verifica se o arquivo foi baixado corretamente**

In [14]:
r = requests.get(url)

# Verificar se o download foi bem-sucedido
if r.status_code == 200:
    # Salvar o arquivo na pasta de destino
    # cria um caminho onde será salvo
    with open(os.path.join(caminho_pasta, "homicídios(1989-2021).csv"), "wb") as f:
        f.write(r.content)
         # 'wb' Se o arquivo existir, ele será sobrescrito se não será criado
    print("Arquivo baixado e salvo com sucesso.")
else:
    print("Falha ao baixar o arquivo.")

Arquivo baixado e salvo com sucesso.


**Cria um dataframe com o arquivo baixado**

In [15]:
Homicidios = pd.read_csv("/content/Base_de_dados/homicídios(1989-2021).csv", sep=",", encoding="UTF-8")

Homicidios

,municipio,estado,período,valor
0,Toncantins,AM,1989,10.41
1,São João do Oriente,MG,1989,0.00
2,São João do Manteninha,MG,1989,0.00
3,São João do Manhuaçu,MG,1989,0.00
4,São João del Rei,MG,1989,8.41
...,...,...,...,...
159211,Primavera do Leste,MT,2021,42.26
159212,Poxoréo,MT,2021,25.10
159213,Porto Estrela,MT,2021,0.00
159214,Porto Esperidião,MT,2021,57.49


**Cria um dicionário para verificar a região**

In [16]:
regioão_por_estado = {
    'AM': 'Norte', 'RR': 'Norte', 'AP': 'Norte', 'PA': 'Norte', 'TO': 'Norte',
    'RO': 'Norte', 'AC': 'Norte', 'MA': 'Nordeste', 'PI': 'Nordeste', 'CE': 'Nordeste',
    'RN': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'AL': 'Nordeste', 'SE': 'Nordeste',
    'BA': 'Nordeste', 'MG': 'Sudeste', 'ES': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'SC': 'Sul', 'RS': 'Sul', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'GO': 'Centro-Oeste', 'DF': 'Centro-Oeste'
}

**Cria um dicionário para renomear as siglas dos estados**

In [17]:
Sigla_estado_completo = {
    'AM': 'Amazonas', 'RR': 'Roraima', 'AP': 'Amapá', 'PA': 'Pará', 'TO': 'Tocantins',
    'RO': 'Rondônia', 'AC': 'Acre', 'MA': 'Maranhão', 'PI': 'Piauí', 'CE': 'Ceará',
    'RN': 'Rio Grande do Norte', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'AL': 'Alagoas', 'SE': 'Sergipe',
    'BA': 'Bahia', 'MG': 'Minas Gerais', 'ES': 'Espírito Santo', 'RJ': 'Rio de Janeiro', 'SP': 'São Paulo',
    'PR': 'Paraná', 'SC': 'Santa Catarina', 'RS': 'Rio Grande do Sul', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
    'GO': 'Goiás', 'DF': 'Distrito Federal'
}


In [18]:
Homicidios['região'] = Homicidios['estado'].map(regioão_por_estado)
Homicidios['estado'] = Homicidios['estado'].map(Sigla_estado_completo)


**Detalha o dataframe**

In [19]:
Homicidios

,municipio,estado,período,valor,região
0,Toncantins,Amazonas,1989,10.41,Norte
1,São João do Oriente,Minas Gerais,1989,0.00,Sudeste
2,São João do Manteninha,Minas Gerais,1989,0.00,Sudeste
3,São João do Manhuaçu,Minas Gerais,1989,0.00,Sudeste
4,São João del Rei,Minas Gerais,1989,8.41,Sudeste
...,...,...,...,...,...
159211,Primavera do Leste,Mato Grosso,2021,42.26,Centro-Oeste
159212,Poxoréo,Mato Grosso,2021,25.10,Centro-Oeste
159213,Porto Estrela,Mato Grosso,2021,0.00,Centro-Oeste
159214,Porto Esperidião,Mato Grosso,2021,57.49,Centro-Oeste


**Transforma o dataframe com regiões para csv **

In [20]:
Homicidios.to_csv("/content/Base_de_dados/homicídios_atualizado.csv", index=False)

In [21]:
# Carregar os dados diretamente para o Spark DataFrame
Homicidios_spark = spark.read.csv('/content/Base_de_dados/homicídios_atualizado.csv', header=True, inferSchema=True)
# calcula a media e arredonda para duas casas decimais
media_geral = Homicidios_spark.select(round(avg('valor'), 2)).first()[0]


In [22]:
#agrupa os dados de acordo com a coluna municipio com 'groupby'

# agg permite após o agrupamento calcular a média

# alias dá um nome para a coluna

# Calcular as médias por município, estado e região
media_municipio = Homicidios_spark.groupBy('municipio').agg(round(avg('valor'), 2).alias('media municipio'))
media_estado = Homicidios_spark.groupBy('estado').agg(round(avg('valor'), 2).alias('media estado'))
media_regiao = Homicidios_spark.groupBy('Região').agg(round(avg('valor'), 2).alias('media regiao'))



**Após o dataframe criado adciona ele ao Homicidios_spark**

In [23]:

# Adicionar as médias calculadas ao DataFrame original
Homicidios_spark = Homicidios_spark.join(media_municipio, on='municipio', how='left')
Homicidios_spark = Homicidios_spark.join(media_estado, on='estado', how='left')
Homicidios_spark = Homicidios_spark.join(media_regiao, on='Região', how='left')


In [24]:
# withCollun cria colunas baseado na condicional "when"

Homicidios_spark = Homicidios_spark.withColumn('acima media Nacional', when(col('valor') > media_geral, 'SIM').otherwise('NÃO'))
Homicidios_spark = Homicidios_spark.withColumn('acima media municipio', when(col('valor') > col('media municipio'), 'SIM').otherwise('NÃO'))
Homicidios_spark = Homicidios_spark.withColumn('acima media estado', when(col('valor') > col('media estado'), 'SIM').otherwise('NÃO'))
Homicidios_spark = Homicidios_spark.withColumn('acima media regiao', when(col('valor') > col('media regiao'), 'SIM').otherwise('NÃO'))

In [25]:
# Mostrar o resultado final
Homicidios_spark.show()

# Converter de volta para pandas para manipulação
Resultado = Homicidios_spark.toPandas()

+-------+------------+--------------------+-------+-----+---------------+------------+------------+--------------------+---------------------+------------------+------------------+
| região|      estado|           municipio|período|valor|media municipio|media estado|media regiao|acima media Nacional|acima media municipio|acima media estado|acima media regiao|
+-------+------------+--------------------+-------+-----+---------------+------------+------------+--------------------+---------------------+------------------+------------------+
|  Norte|    Amazonas|          Toncantins|   1989|10.41|           2.47|       10.45|       17.36|                 NÃO|                  SIM|               NÃO|               NÃO|
|Sudeste|Minas Gerais| São João do Oriente|   1989|  0.0|           7.99|        9.35|       10.72|                 NÃO|                  NÃO|               NÃO|               NÃO|
|Sudeste|Minas Gerais|São João do Mante...|   1989|  0.0|           16.1|        9.35|       10

In [26]:
Resultado.to_csv("/content/Base_de_dados/homicídios(1989-2021)_FINAL.csv", sep = ";",encoding="UTF-8",index=False, decimal=",")

**FAZ DOWNLOAD DO ARQUIVO**

In [27]:
files.download("/content/Base_de_dados/homicídios(1989-2021)_FINAL.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>